In [1]:
%load_ext autoreload
%autoreload 2

from google.colab import drive
drive.mount('/content/drive')

import plotly.express as px
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns
from tqdm.notebook import trange, tqdm

import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers.schedules import InverseTimeDecay
from tensorflow.keras.optimizers import Adam

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error

!pip install wandb -qqq
import wandb
from wandb.keras import WandbCallback
wandb.login()

import sys
price_predictor = '/content/drive/MyDrive/1 Projects/bitcoin_price_predictor/price_predictor'
sys.path.append(price_predictor)

from helpers import load_close_data, get_n_test_samples, \
                    repeat_evaluate, summarize_scores, train_and_validate

DOWNLOAD_DIR = Path('../download')
DOWNLOAD_DIR = Path('/content/drive/MyDrive/1 Projects/bitcoin_price_predictor/download')
DATA_DIR = Path('/content/drive/MyDrive/1 Projects/bitcoin_price_predictor/data')

sns.set()

Mounted at /content/drive
     |████████████████████████████████| 2.1MB 5.6MB/s 
     |████████████████████████████████| 133kB 46.9MB/s 
     |████████████████████████████████| 163kB 39.2MB/s 
     |████████████████████████████████| 102kB 11.0MB/s 
     |████████████████████████████████| 71kB 8.3MB/s 


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [2]:
data = load_close_data(DOWNLOAD_DIR, dropna=True)
data.shape

(92192,)

# Manual LR Scheduling

In [4]:
for i in range(5):
    print(f'RUN NUMBER {i}')
    run = wandb.init(project='bitcoin_price_predictor',
                    config={
                        'dataset': 1,
                        # log, log_and_divide_a, log_and_range_a_b
                        'scaler': 'log_and_range_0_1',
                        'n_input': 168, # num lag observations
                        'n_nodes': 300, # num nodes per lauyer
                        'n_epochs': 150, # num training epochs
                        'n_batch': 168 * 20, # batch size
                        'num_layers': 3, # num layers used in MLP
                        'model_type': 'MLP',
                        'activation': 'relu',
                        'loss': 'mse',
                        # Model LR scheduler and optimizer
                        'use_lr_scheduler': False,
                        'lr': 0.1,
                        'optimizer': 'adam',
                        # Other
                        'verbose': 0, # control verbosity of Keras fit
                        'dropna': True # whether to drop missing values from data
                            })
    config = wandb.config # we use this to configure our experiment

    history = train_and_validate(config)

wandb: Currently logged in as: theadammurphy (use `wandb login --relogin` to force relogin)


RUN NUMBER 0


KeyboardInterrupt: ignored